# E. coli Tryptophan Network Construction
## Glucose Growth

In [1]:
# Disable gurobi logging output for this notebook.
try:
    import gurobipy
    gurobipy.setParam("OutputFlag", 0)
except ImportError:
    pass

import numpy as np

import pathlib
import pandas as pd
import altair as alt
import sympy as sym

import mass
from mass import MassModel, MassReaction, MassMetabolite, Simulation
from mass.io.sbml import write_sbml_model as write_mass_sbml_model

import cobra
from cobra.io.sbml import read_sbml_model as read_cobra_sbml_model

print(f"COBRApy version: {cobra.__version__}")
print(f"MASSpy version: {mass.__version__}")

Set parameter Username
Academic license - for non-commercial use only - expires 2022-01-21
COBRApy version: 0.22.1
MASSpy version: 0.1.5


## Set paths and constants

In [2]:
from cobra import Configuration
config = Configuration()
config.solver = "gurobi"

## Create Model of Tryptophan Network
### Load COBRA model

In [3]:
cobra_model = read_cobra_sbml_model(f"./models/cobra/iML1515.xml")

### View network map via Escher

In [4]:
import escher

In [5]:
escher_builder = escher.Builder(
    model=cobra_model,
    map_json="./maps/Tryptophan_map.json",
    highlight_missing=True) 

escher_builder

Builder(highlight_missing=True)

## Obtain Flux State
### Load flux growth data

In [6]:
medium = "Glucose"

In [7]:
flux_data = pd.read_excel(
    io="./data/growth_data.xlsx",
    sheet_name="flux_data",
    index_col=0
)
flux_data = flux_data.loc[lambda x: x['Growth Medium'] == medium]
flux_data = flux_data.drop("Growth Medium", axis=1)
flux_data

,Flux (mmol * gDW-1 * h-1)
ID,
EX_ac_e,6.827019e+00
ACt2rpp,-6.827019e+00
ACKr,-6.827019e+00
PTAr,6.827019e+00
EX_glc__D_e,-9.654000e+00
GLCptspp,9.654000e+00
EX_fum_e,9.653732e-08
PGI,5.699970e+00
PFK,7.058477e+00


### Knockout reactions


In [8]:
#Knocking out TRPS2 and TRPS3
cobra_model.reactions.TRPS2.knock_out()
cobra_model.reactions.TRPS3.knock_out()
cobra_model.reactions.TRPAS2.knock_out()

### Formulate QP minimization for known fluxes and optimize

In [9]:
v_vars = []
v_data = []

# For irreversible enzyme pairs, flux data is given as Enzyme1 - Enzyme2 = value.
# To ensure all enzymes have some flux, add a percentage of the net flux for each enzyme
# The netflux will still remain the same value.
irreversible_enzyme_pairs = [["PFK", "FBP"], ["PYK", "PPS"]]
reverse_flux_percent = 0.1


for rid, flux in flux_data.itertuples():
#     if rid == "BIOMASS_Ec_iML1515_core_75p37M":
#         reaction = cobra_model.reactions.get_by_id(rid)
#         reaction.bounds = (flux, flux)
    # Ensure bounds are set to allow import from growth medium 
    if (rid == "EX_glc__D_e" and medium == "Glucose") or\
       (rid == "EX_pyr_e" and medium == "Pyruvate"):
        reaction = cobra_model.reactions.get_by_id(rid)
        reaction.bounds = ({"Glucose": -10000, "Pyruvate": -1000}.get(medium), 1000)
    # Make adjustments to net flux of PFK/FBP and PYK/PPS to ensure
    # no target flux value is 0 in order to create an enzyme module.
    for irreversible_enzyme_pair in irreversible_enzyme_pairs:
        if rid in irreversible_enzyme_pair:
            flux1, flux2 = flux_data.loc[irreversible_enzyme_pair, "Flux (mmol * gDW-1 * h-1)"].values
            if flux1 == 0:
                flux += reverse_flux_percent * flux2 # mmol*gDW^-1*hr^-1
            if flux2 == 0:
                flux += reverse_flux_percent * flux1 # mmol*gDW^-1*hr^-1
            print(rid, flux)
    v_vars.append(sym.Symbol(rid))
    v_data.append(flux)

# Make symbolic for optlang objective 
v_vars = sym.Matrix(v_vars)
v_data = sym.Matrix(v_data)
F = sym.Matrix(2 * sym.eye(len(v_vars)))

objective = 0.5 * v_vars.T * F * v_vars  - (2 * v_data).T * v_vars
cobra_model.objective = objective[0]
cobra_model.objective_direction = "min"

flux_solution = cobra_model.optimize()

PFK 7.76432470140912
FBP 0.7058477001281019
PYK 2.7360389076214697
PPS 0.24873080978377


#### Compare fluxes

In [10]:
data = pd.concat(objs=(flux_data, flux_solution.fluxes), axis=1).dropna()
data = data.reset_index()
data.columns = ["Reaction ID", "Measured", "Computed"]

flux_compare = alt.Chart(
    data=data,
    width=300,
    height=300,
).mark_circle(size=60).encode(
    x='Measured:Q',
    y='Computed:Q',
    color='Reaction ID',
    tooltip=list(data.columns)
)
line = pd.DataFrame({
    'Measured': [-20, 20],
    'Computed':  [-20, 20],
})
line_plot = alt.Chart(line).mark_line(color= 'grey').encode(
    x='Measured',
    y='Computed',
)
flux_compare += line_plot
flux_compare

alt.LayerChart(...)

In [11]:
flux_solution["TRPt2rpp"]

0.0

### Create MASS model


In [12]:
# Create MassModel
mass_model = MassModel("Tryptophan", array_type="DataFrame")

# Reactions to extract into subnetwork
reaction_list = [
    "TRPt2rpp",
    "GLCptspp",
    "PGI", 
    "PFK", 
    "FBP", 
    "FBA", 
    "TPI", 
    "GAPD",
    "PGK", 
    "PGM", 
    "ENO", 
    "PYK", 
    "PPS", 
    #"LDH_D",
    "PGCD",
    "PSERT",
    "PSP_L",
    "CHORS",
    "DDPA", 
    "DHQS",
    "DHQTi",
    "PSCVT",
    "SHK3Dr",
    "SHKK", 
    "TALA", 
    "G6PDH2r",
    "GND",
    "RPI",
    "TKT1", 
    "TKT2", 
    "ANS", 
    "ANPRT", 
    "IGPS",
    "PRAIi",
    "TRPS1",
    #"TRPS3",
    #"TRPS2",
    "RPE",
    "PGL",
    ]


cobra_reactions = cobra_model.reactions.get_by_any(reaction_list)
mass_model.add_reactions([MassReaction(rxn) for rxn in cobra_reactions])
mass_model

Name,Tryptophan
Memory address,0x07f7b98bfa160
Stoichiometric Matrix,51x36
Matrix Rank,36
Number of metabolites,51
Initial conditions defined,0/51
Number of reactions,36
Number of genes,34
Number of enzyme modules,0
Number of groups,0
Objective expression,0


#### Convert units

In [13]:
growth_rate = flux_data.loc[cobra_model.reactions.BIOMASS_Ec_iML1515_core_75p37M.id][0]

In [14]:
T = 313.15
gas_constant = 0.00831446261815324
e_coli_density = 1.1 # g / mL assumption
volume = 3.2  # femtoliter
# Perform conversions
doubling_time_per_minute = np.log(2) / growth_rate * 60
cell_gDW = 41932 * doubling_time_per_minute**-1.232 * 1e-15
real_cell_total_weight = e_coli_density * (volume * 1e-12) # fL --> mL
# Assume water is 70%
adj_volume = volume * 0.7
gDW_L_conversion_factor = real_cell_total_weight / (adj_volume * 1e-15)
gDW_L_conversion_factor

1571.4285714285716

#### Set fluxes

In [15]:
for reaction in mass_model.reactions.get_by_any(reaction_list):
    flux = flux_solution[reaction.id]
    if reaction.id == "TRPt2rpp" and flux == 0:
        flux+= 0.000001
    # Convert mmol * gDW-1 * h-1 --> Mole / L / hr
    reaction.steady_state_flux = flux * gDW_L_conversion_factor * 0.001
    print(f"{reaction.id}: {flux}")

TRPt2rpp: 1e-06
GLCptspp: 9.648697724650988
PGI: 5.699970323068908
PFK: 7.76432470140912
FBP: 0.7058477001281019
FBA: 7.058477001280537
TPI: 6.8222429539041745
GAPD: 15.7103918012915
PGK: -15.7103918012915
PGM: -14.563006476638412
ENO: 14.563006476638412
PYK: 2.730736632272458
PPS: 0.2540330851327813
PGCD: 1.1473853246530883
PSERT: 1.1473853246530883
PSP_L: 1.1473853246530883
CHORS: 0.21777740347048
DDPA: 0.21777740347048
DHQS: 0.21777740347048
DHQTi: 0.21777740347048
PSCVT: 0.21777740347048
SHK3Dr: 0.21777740347048
SHKK: 0.21777740347048
TALA: 0.7977394783488181
G6PDH2r: 3.919275276928566
GND: 2.8322842173013867
RPI: -1.434252429002472
TKT1: 0.797739478348818
TKT2: 0.5798475340021348
ANS: 0.029196623435081406
ANPRT: 0.029196623435081406
IGPS: 0.029196623435081406
PRAIi: 0.029196623435081406
TRPS1: 0.029196623435081406
RPE: 1.3775870123509528
PGL: 3.919275276928566


### Obtain Concentrations

#### Set equilibrium constants


In [16]:
Keq_data = pd.read_excel(
    io="./data/growth_data.xlsx",
    sheet_name="Keq_data",
    index_col=0
)

Keq_data = Keq_data.drop("Stoichiometry", axis=1)
# Set equilibrium constants
for reaction in mass_model.reactions.get_by_any(reaction_list):
    try:
        Keq = Keq_data.loc[reaction.id, :][0]
        reaction.equilibrium_constant = Keq 
    except KeyError as e:
        print(f"No Keq data for {e}")
        if reaction.id == "TRPt2rpp":
            reaction.equilibrium_constant = 1
    
    print(f"{reaction.Keq_str}: {reaction.equilibrium_constant}")

No Keq data for 'TRPt2rpp'
Keq_TRPt2rpp: 1
Keq_GLCptspp: 52155672.97
Keq_PGI: 0.372679859
Keq_PFK: 1022.334096
Keq_FBP: 64.75642481
Keq_FBA: 0.000102656
Keq_TPI: 0.087276143
Keq_GAPD: 0.627040514
Keq_PGK: 0.000670855
Keq_PGM: 2.981497079
Keq_ENO: 6.69595445
Keq_PYK: 17602.30756
Keq_PPS: 2.209787365
Keq_PGCD: 7.58e-06
Keq_PSERT: 52.82458434
Keq_PSP_L: 27.02318343
Keq_CHORS: 134000000000.0
Keq_DDPA: 2340000000000.0
Keq_DHQS: 3.46e+17
Keq_DHQTi: 7.045333375
Keq_PSCVT: 770.0
Keq_SHK3Dr: 15.9
Keq_SHKK: 32000.0
Keq_TALA: 1.1899999992265
Keq_G6PDH2r: 15.663669
Keq_GND: 0.050066728
Keq_RPI: 0.579698787
Keq_TKT1: 2.589775912
Keq_TKT2: 28.10232048
Keq_ANS: 1.13e+16
Keq_ANPRT: 2190000000000.0
Keq_IGPS: 220000000000.0
Keq_PRAIi: 38.71017608
Keq_TRPS1: 135031.8001
Keq_RPE: 1.979534309
Keq_PGL: 35988.51087


In [17]:
# Potentially missing
['ara5p_c', 'indole_c']

['ara5p_c', 'indole_c']

### Set concentrations

In [18]:
conc_data = pd.read_excel(
    io="./data/growth_data.xlsx",
    sheet_name="conc_data",
    index_col=0
)
conc_data = conc_data.loc[lambda x: x['Growth Medium'] == "Glucose"]
conc_data = conc_data.drop("Growth Medium", axis=1)
conc_data = conc_data.drop("Name", axis=1)
conc_data

,Concentration (mol * L-1)
ID,
g6p_c,0.000843
13dpg_c,0.000045
2pg_c,0.000150
3pg_c,0.000500
6pgc_c,0.000247
adp_c,0.000708
akg_c,0.000532
amp_c,0.000428
anth_c,0.000003


#### Add metabolites in enzymes that not present in MASS model
This includes ``indole_c`` and `ara5p_c`

In [19]:
missing_metabolites = ["indole_c", "ara5p_c"]
for mid in missing_metabolites:
    metabolite = MassMetabolite(cobra_model.metabolites.get_by_id(mid))
    metabolite.fixed = True
    mass_model.add_metabolites([metabolite])

In [20]:
for metabolite in ["h2o_c", "h_c"]:
    metabolite = mass_model.metabolites.get_by_id(metabolite)
    metabolite.fixed = True
    metabolite.initial_condition = 1

In [21]:
mass_model.update_initial_conditions({
    mid: value for mid, value in conc_data.itertuples()
})
missing_ics = mass_model.metabolites.query(lambda m: m.initial_condition is None)
missing_ics

[<MassMetabolite h_p at 0x7f7b78f5bbb0>,
 <MassMetabolite trp__L_p at 0x7f7baa20c550>,
 <MassMetabolite glc__D_p at 0x7f7baa20c9d0>,
 <MassMetabolite 3php_c at 0x7f7baa1f9640>,
 <MassMetabolite pser__L_c at 0x7f7b78f5b910>,
 <MassMetabolite 3psme_c at 0x7f7bb9483d00>,
 <MassMetabolite chor_c at 0x7f7bb9483d60>,
 <MassMetabolite 2dda7p_c at 0x7f7bb9483e50>,
 <MassMetabolite 3dhq_c at 0x7f7bb947b700>,
 <MassMetabolite 3dhsk_c at 0x7f7bb947bf40>,
 <MassMetabolite skm5p_c at 0x7f7bb947b400>,
 <MassMetabolite 6pgl_c at 0x7f7bb947b5b0>,
 <MassMetabolite pran_c at 0x7f7baa1f5cd0>,
 <MassMetabolite 2cpr5p_c at 0x7f7baa1f5e20>,
 <MassMetabolite 3ig3p_c at 0x7f7baa1f5ee0>,
 <MassMetabolite indole_c at 0x7f7baa1c2fd0>,
 <MassMetabolite ara5p_c at 0x7f7baa1bc220>]

In [22]:
# Provide initial guesses
# glc__D_p and ara5p_c are 
for metabolite in missing_ics:
    if metabolite.id in ['anth_c', 'glc__D_p', "trp__L_p", "ara5p_c"]:
        metabolite.initial_condition = 0.001
    if metabolite.id in ["indole_c"]:
        metabolite.initial_condition = 0.000001
missing_ics = mass_model.metabolites.query(lambda m: m.initial_condition is None)
missing_ics

[<MassMetabolite h_p at 0x7f7b78f5bbb0>,
 <MassMetabolite 3php_c at 0x7f7baa1f9640>,
 <MassMetabolite pser__L_c at 0x7f7b78f5b910>,
 <MassMetabolite 3psme_c at 0x7f7bb9483d00>,
 <MassMetabolite chor_c at 0x7f7bb9483d60>,
 <MassMetabolite 2dda7p_c at 0x7f7bb9483e50>,
 <MassMetabolite 3dhq_c at 0x7f7bb947b700>,
 <MassMetabolite 3dhsk_c at 0x7f7bb947bf40>,
 <MassMetabolite skm5p_c at 0x7f7bb947b400>,
 <MassMetabolite 6pgl_c at 0x7f7bb947b5b0>,
 <MassMetabolite pran_c at 0x7f7baa1f5cd0>,
 <MassMetabolite 2cpr5p_c at 0x7f7baa1f5e20>,
 <MassMetabolite 3ig3p_c at 0x7f7baa1f5ee0>]

In [23]:
from mass.thermo import (
    ConcSolver, sample_concentrations,
    update_model_with_concentration_solution)

In [24]:
conc_solver = ConcSolver(
    mass_model,
    excluded_metabolites=["h_c", "h2o_c"],
    equilibrium_reactions=[
        r.id for r in mass_model.reactions
        if r.steady_state_flux == 0],
    constraint_buffer=1,
    zero_value_log_substitute=1e-7,

)

conc_solver.setup_feasible_qp_problem(
    fixed_conc_bounds=list(mass_model.fixed),
    fixed_Keq_bounds=mass_model.reactions.list_attr("Keq_str")
)

conc_solution = conc_solver.optimize()
conc_solution

,variables,reduced_costs
h_p,2.692905e-02,0.000000
trp__L_p,1.000000e-03,0.000000
trp__L_c,9.906642e-06,0.000000
glc__D_p,1.000000e-03,0.000000
pep_c,2.601339e-04,0.000000
...,...,...
Keq_IGPS,2.200000e+11,0.000000
Keq_PRAIi,3.871018e+01,0.000000
Keq_TRPS1,1.350318e+05,0.000000
Keq_RPE,1.979534e+00,-0.317674


In [25]:
data = pd.concat(objs=(conc_data, conc_solution.concentrations), axis=1).dropna()
data = data.reset_index()
data.columns = ["Metabolite ID", "Measured", "Computed"]

conc_compare = alt.Chart(
    data=data,
    width=300,
    height=300,
).mark_circle(size=60).encode(
    alt.X('Measured', type='quantitative', scale=alt.Scale(type="log")),
    alt.Y('Computed', type='quantitative', scale=alt.Scale(type="log")),
    color='Metabolite ID',
    tooltip=list(data.columns)
)
line = pd.DataFrame({
    'Measured': [1e-7, .1],
    'Computed':  [1e-7, .1],
})
line_plot = alt.Chart(line).mark_line(color= 'grey').encode(
    x='Measured',
    y='Computed',
)
conc_compare += line_plot
conc_compare

alt.LayerChart(...)

#### Sample Concentrations

In [26]:
n_models = 10

In [27]:
# Fix Metabolite IDs as SBML compatible
for metabolite in mass_model.metabolites:
    if metabolite.id[0].isdigit():
        metabolite.id = f"_{metabolite.id}"
mass_model.repair()

In [28]:
conc_solver = ConcSolver(
    mass_model,
    excluded_metabolites=["h_c", "h2o_c"],
    equilibrium_reactions=[
        r.id for r in mass_model.reactions
        if r.steady_state_flux == 0],
    constraint_buffer=1,

)

conc_solver.setup_sampling_problem(
    fixed_conc_bounds=list(mass_model.fixed),
    fixed_Keq_bounds=mass_model.reactions.list_attr("Keq_str"))
for variable in conc_solver.variables:
    try:
        met = mass_model.metabolites.get_by_id(variable.name)
        variable.lb, variable.ub = np.log([met.ic / 10, met.ic * 10])
    except:
        pass
conc_samples = sample_concentrations(conc_solver, n=n_models, seed=4)
conc_samples

,h_p,trp__L_p,trp__L_c,glc__D_p,pep_c,g6p_c,pyr_c,f6p_c,atp_c,adp_c,...,xu5p__D_c,gln__L_c,anth_c,prpp_c,ppi_c,pran_c,_2cpr5p_c,_3ig3p_c,indole_c,ara5p_c
0,0.001640,0.007554,0.000005,0.001190,0.000094,0.008184,0.000656,0.000257,0.003485,0.000072,...,0.000416,0.000696,1.303589e-06,0.000099,0.000090,1.385677e-07,1.354548e-08,1.375337e-09,1.008793e-07,0.000100
1,0.001359,0.007358,0.000004,0.001045,0.000175,0.006960,0.000656,0.000264,0.007163,0.000417,...,0.000536,0.001023,8.304599e-07,0.000045,0.000054,2.079298e-05,1.690062e-05,5.529732e+00,2.980551e-07,0.000138
2,0.010855,0.003649,0.000015,0.003324,0.000121,0.001534,0.000470,0.000145,0.001829,0.000192,...,0.000377,0.001487,5.040581e-07,0.000024,0.000042,1.115536e-07,4.070078e-09,9.364868e+01,3.018352e-07,0.000146
3,0.001334,0.004984,0.000002,0.000282,0.000083,0.002176,0.000274,0.000264,0.004080,0.000145,...,0.000674,0.004339,1.161803e-06,0.000032,0.000058,5.224638e-07,5.119618e-07,9.261001e+06,5.970943e-07,0.000166
4,0.001425,0.002747,0.000001,0.000236,0.000070,0.001482,0.000314,0.000197,0.001020,0.000086,...,0.000344,0.002318,7.376815e-07,0.000022,0.000043,1.556940e-06,2.875676e-08,5.760793e+02,3.821237e-07,0.000118
5,0.002896,0.007679,0.000008,0.000821,0.000129,0.006851,0.000638,0.000373,0.002642,0.000072,...,0.000858,0.004607,1.759109e-06,0.000060,0.000043,1.341755e-06,1.583892e-07,6.355009e+07,6.193466e-07,0.000242
6,0.012942,0.002133,0.000010,0.001174,0.000345,0.002630,0.000363,0.000295,0.001338,0.000150,...,0.000299,0.001676,1.271133e-06,0.000045,0.000057,2.980207e-10,5.786709e-10,8.925965e+00,3.682893e-07,0.000220
7,0.012024,0.004135,0.000018,0.002226,0.000378,0.004974,0.000079,0.000133,0.002573,0.000209,...,0.000324,0.003901,6.110919e-06,0.000082,0.000094,1.906148e-09,8.776489e-10,2.605393e+02,4.632259e-07,0.000241
8,0.001825,0.002316,0.000002,0.000672,0.000239,0.005937,0.000108,0.000278,0.006639,0.000236,...,0.000400,0.005616,6.759271e-06,0.000110,0.000090,2.556527e-09,2.311143e-10,1.746400e+06,6.714419e-07,0.000222
9,0.005002,0.000992,0.000002,0.000560,0.000289,0.004642,0.000236,0.000412,0.013033,0.000719,...,0.000311,0.001776,3.335338e-06,0.000155,0.000053,1.107933e-07,1.654566e-07,6.009055e+00,3.329448e-07,0.000105


May take a few minutes to run

In [29]:
models_for_ensemble = []
for idx, conc_sample in conc_samples.iterrows():
    # Make copy of new model
    new_model = mass_model.copy()
    new_model.id += "_C{0:d}".format(idx)
    print(f"Creating model {new_model.id}")
    # Get concentration sample and update model with sample
    new_model.update_initial_conditions(conc_sample.to_dict())
    fluxes = np.array(list(new_model.steady_state_fluxes.values()))
    imbalanced_metabolites = new_model.S.dot(fluxes)
    # Iterate through metabolites
    for mid, imbalance in imbalanced_metabolites.iteritems():
        # Ignore balanced metabolites
        if imbalance == 0:
            continue
        # Get metabolite object
        met = new_model.metabolites.get_by_id(mid)

        # Add boundary reactions for imbalanced metabolites
        boundary_type = "sink"    
        # Add boundary reaction with imbalance as flux value
        boundary_reaction = new_model.add_boundary(
            mid, boundary_type, boundary_condition=met.ic)

        boundary_reaction.Keq = 1
        if imbalance < 0:
            boundary_reaction.reverse_stoichiometry(inplace=True)
            imbalance = -imbalance

        boundary_reaction.kf = imbalance / met.ic
        boundary_reaction.steady_state_flux = imbalance
        try:
            # Update PERCs
            new_model.calculate_PERCs(
                fluxes={
                    r: v for r, v in new_model.steady_state_fluxes.items()
                    if not r.boundary},
                update_reactions=True)
        except:
            print("Negative PERCs for {0}".format(new_model.id))
            continue
    models_for_ensemble.append(new_model)
print("Number of models in ensemble: {0:d}".format(
    len(models_for_ensemble)))

Creating model Tryptophan_C0
Creating model Tryptophan_C1
Creating model Tryptophan_C2
Creating model Tryptophan_C3
Creating model Tryptophan_C4
Creating model Tryptophan_C5
Creating model Tryptophan_C6
Creating model Tryptophan_C7
Creating model Tryptophan_C8
Creating model Tryptophan_C9
Number of models in ensemble: 10


In [30]:
from mass.io.json import save_json_model

In [31]:
models_to_export = models_for_ensemble
for model in models_to_export:
    save_json_model(
        mass_model=model,
        filename=f"./models/mass/without_enzymes/{model.id}.json")
    
print("Number of models exported: {0:d}".format(len(models_to_export)))

Number of models exported: 10


### Create Enzyme Modules

In [32]:
from enzyme_construction import make_enzyme_module_from_dir

In [33]:
isozyme1_percent = 0.75
isozyme2_percent = 1 - isozyme1_percent


# Isozymes and flux split percentages,
isozymes_and_flux_splits = {
#     "PFK": {
#         "PFK1": isozyme1_percent,
#         "PFK2": isozyme2_percent,
#     },
#     "FBP": {
#         "FBP1": isozyme1_percent,
#         "FBP2": isozyme2_percent,
#     },
#     "FBA": {
#         "FBA1": isozyme1_percent,
#         "FBA2": isozyme2_percent,
#     },
#     "PGM": {
#         "PGMi": isozyme1_percent,
#         "PGMd": isozyme2_percent,
#     },
    "PYK": {
        "PYK1": isozyme1_percent,
        "PYK2": isozyme2_percent,
    },
}
        
isozymes_and_flux_splits

{'PYK': {'PYK1': 0.75, 'PYK2': 0.25}}

In [34]:
for new_model in models_for_ensemble.copy():
    enzyme_modules = []
    for reaction in new_model.reactions.get_by_any(reaction_list):
        # Flip reaction flux for those that have opposite stoichiometry from Enzyme Data
        if reaction.id in ["PGK", "PGM"]:
            flux = -reaction.steady_state_flux
        else:
            flux = reaction.steady_state_flux

        # Make isozymes
        if reaction.id in isozymes_and_flux_splits:

            isozymes_and_flux_split = isozymes_and_flux_splits[reaction.id]

            for isozyme, flux_split in isozymes_and_flux_split.items():
                enzyme_module = make_enzyme_module_from_dir(
                    enzyme_id=isozyme,
                    steady_state_flux=flux * flux_split, # Split flux for isozymes
                    metabolite_concentrations=new_model.initial_conditions,
                    path_to_dir="./data/enzyme_data",
                    kcluster=1,
                    enzyme_gpr=None,
                    zero_tol=1e-15)
                enzyme_modules += [enzyme_module]
    new_model.remove_reactions([reaction])
    for enzyme_module in enzyme_modules:
        new_model = new_model.merge(enzyme_module, inplace=True)
    print(f"finished {new_model}")

finished Tryptophan_C0
finished Tryptophan_C1
finished Tryptophan_C2
finished Tryptophan_C3
finished Tryptophan_C4
finished Tryptophan_C5
finished Tryptophan_C6
finished Tryptophan_C7
finished Tryptophan_C8
finished Tryptophan_C9


In [35]:
# Ensure all models are simulated to a long time and that
# there are enough time points to determine if model reached steady state.
models_to_export = []
tfinal = 1e6
sim = Simulation(reference_model=models_for_ensemble[0])
sim.integrator.maximum_time_step = 100
sim.add_models(models_for_ensemble[1:], disable_safe_load=True)
sim.integrator.absolute_tolerance = 1e-15
sim.integrator.relative_tolerance = 1e-12
for model in models_for_ensemble.copy():
    # Attempt to determine steady state
    conc_sol, flux_sol = sim.find_steady_state(
        models=model, strategy="simulate", update_values=True,
        tfinal=tfinal)
    if conc_sol and flux_sol:
        continue
    print(str(model) + " attempt failed.")
    models_to_export.append(model)
    

mass/simulation/simulation.py:900 UserWarning: Unable to find a steady state for one or more models. Check the log for more details.
ERROR: Unable to find a steady state for 'Tryptophan_C0' using strategy 'simulate' due to the following: For MassModel "Tryptophan_C0", absolute difference for "['[chor_c]', '[_3dhq_c]', '[_3dhsk_c]']" is greater than the steady state threshold.


Tryptophan_C0 attempt failed.


mass/simulation/simulation.py:900 UserWarning: Unable to find a steady state for one or more models. Check the log for more details.
ERROR: Unable to find a steady state for 'Tryptophan_C3' using strategy 'simulate' due to the following: For MassModel "Tryptophan_C3", absolute difference for "['[_3ig3p_c]']" is greater than the steady state threshold.


Tryptophan_C3 attempt failed.


mass/simulation/simulation.py:900 UserWarning: Unable to find a steady state for one or more models. Check the log for more details.
ERROR: Unable to find a steady state for 'Tryptophan_C5' using strategy 'simulate' due to the following: For MassModel "Tryptophan_C5", absolute difference for "['[_3dhq_c]', '[_3dhsk_c]', '[_3ig3p_c]']" is greater than the steady state threshold.


Tryptophan_C5 attempt failed.


mass/simulation/simulation.py:900 UserWarning: Unable to find a steady state for one or more models. Check the log for more details.
ERROR: Unable to find a steady state for 'Tryptophan_C8' using strategy 'simulate' due to the following: For MassModel "Tryptophan_C8", absolute difference for "['[_3ig3p_c]']" is greater than the steady state threshold.


Tryptophan_C8 attempt failed.


In [36]:
models_to_export = sim.get_model_objects(sim.models)
models_to_export = models_for_ensemble
for model in models_to_export:
    save_json_model(
        mass_model=model,
        filename=f"./models/mass/with_enzymes/{model.id}.json")
    
print("Number of models exported: {0:d}".format(len(models_to_export)))

Number of models exported: 10
